In [40]:
# 1) 기존 꼬인 것들 싹 제거
%pip uninstall -y langchain langchain-core langchain-text-splitters langchain-google-genai langchain-community langchain_community langchain-openai

# 2) 필요한 것만 딱 맞는 버전으로 재설치
%pip install "langchain-core>=0.3.27,<0.4.0" "langchain-text-splitters>=0.3.0,<0.4.0" "langchain-google-genai==2.0.10"


Note: you may need to restart the kernel to use updated packages.


  Using cached langchain_core-0.3.79-py3-none-any.whl.metadata (3.2 kB)
  Using cached langchain_google_genai-2.0.10-py3-none-any.whl.metadata (3.6 kB)
Using cached langchain_google_genai-2.0.10-py3-none-any.whl (41 kB)
Using cached langchain_core-0.3.79-py3-none-any.whl (449 kB)

   ---------------------------------------- 0/3 [langchain-core]
   ---------------------------------------- 0/3 [langchain-core]
   ---------------------------------------- 0/3 [langchain-core]
   ---------------------------------------- 0/3 [langchain-core]
   ---------------------------------------- 0/3 [langchain-core]
   ---------------------------------------- 0/3 [langchain-core]
   ---------------------------------------- 0/3 [langchain-core]
   ---------------------------------------- 0/3 [langchain-core]
   ---------------------------------------- 0/3 [langchain-core]
   ---------------------------------------- 0/3 [langchain-core]
   ---------------------------------------- 0/3 [langchain-core]
   

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain-classic 1.0.0 requires langchain-text-splitters<2.0.0,>=1.0.0, but you have langchain-text-splitters 0.3.11 which is incompatible.
langgraph-prebuilt 1.0.4 requires langchain-core>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: C:\Users\jskim\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# 환경 설정
| 패키지                          | 버전                                 |
| ---------------------------- | ---------------------------------- |
| **langchain-core**           | `0.3.x` (권장: `0.3.27` 이상 `<0.4.0`) |
| **langchain-text-splitters** | `0.3.x`                            |
| **langchain-google-genai**   | `2.0.10`                           |
| **google-generativeai**      | `>=1.0.0`                          |
| **dotenv**                   | 최신                                 |


In [2]:
# ============================================
# 1. 환경설정 & 라이브러리 import
# ============================================
import os
import json
import ast
from pathlib import Path
from typing import List

from dotenv import load_dotenv

from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.prompts import ChatPromptTemplate

from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    GoogleGenerativeAIEmbeddings,
)

In [3]:
# ============================================
# 2. API 키 로드 (.env)
# ============================================
load_dotenv() 

if not os.environ.get("GOOGLE_API_KEY"):
    raise ValueError("GOOGLE_API_KEY 가 설정되어 있지 않습니다. .env 파일을 확인하세요.")

# 3. 경로 설정

BASE_DIR = Path.cwd()
DATA_PATH = BASE_DIR / "meta_data_contents_summary.json"


# 데이터 로딩

이 단계에서는 JSON의 contents_summary 필드를 ast.literal_eval로 파싱하여
구조화된 요약 텍스트만으로 LangChain Document를 생성한다.

In [4]:
# ============================================
# 3. 데이터 로드 (요약 기반 loader)
# ============================================

def load_and_prepare_docs(filepath: Path) -> List[Document]:
    """
    meta_data_contents_summary.json을 읽어서
    'contents_summary' 기반의 요약 텍스트만으로 LangChain Document 리스트를 만든다.
    """
    print(f"[load_and_prepare_docs] Loading from: {filepath}")
    with open(filepath, "r", encoding="utf-8") as f:
        audit_cases = json.load(f)

    docs: List[Document] = []

    for i, case in enumerate(audit_cases):
        site = case.get("site", "알 수 없음")
        category = case.get("category", "알 수 없음")
        date = case.get("date", "알 수 없음")
        original_title = case.get("title", "")

        metadata = {
            "index": i,
            "title": original_title,
            "site": site,
            "category": category,
            "date": date,
        }

        # contents_summary는 문자열 형태의 dict
        summary_str = case.get("contents_summary")
        summary_dict = {}
        if summary_str:
            try:
                summary_dict = ast.literal_eval(summary_str)
            except Exception:
                summary_dict = {}

        title = summary_dict.get("title_str", original_title)
        keywords = ", ".join(summary_dict.get("keyword_list", []))
        problems = summary_dict.get("problems_str", "")
        action = summary_dict.get("action_str", "")
        standards = summary_dict.get("standards_str", "")

        summary_based_text = (
            f"출처: {site}\n"
            f"분류: {category}\n"
            f"일자: {date}\n"
            f"제목: {title}\n"
            f"핵심 키워드: {keywords}\n"
            f"문제 요약: {problems}\n"
            f"조치 요약: {action}\n"
            f"관련 규정: {standards}"
        )

        docs.append(Document(page_content=summary_based_text, metadata=metadata))

    print(f"[load_and_prepare_docs] Created {len(docs)} summary-based documents.")
    return docs



# Retriever 구현
RAG의 핵심 단계이며 다음 순서를 따른다:

1. 문서 청크 분할

    RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)

2. Gemini 임베딩 생성

    공식 권장 모델: models/gemini-embedding-001

3. InMemoryVectorStore 인덱싱

4. vector_store.as_retriever(k=4) 로 변환

    → LangChain Runnable과 호환되는 retriever 객체 완성

In [5]:
# ============================================
# 4. Retriever 구현 (Gemini 임베딩 사용)
# ============================================
def build_retriever(docs: List[Document]):
    # 1) 청크 단위로 자르기 (요약이라 짧긴 하지만 일단 통일)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
        separators=["\n\n", "\n", " ", ""],
    )
    splits = text_splitter.split_documents(docs)
    print(f"[build_retriever] Split into {len(splits)} chunks.")

    # 2) Gemini 임베딩
    embeddings = GoogleGenerativeAIEmbeddings(
        model="models/text-embedding-004",
    )

    # 3) In-memory Vector Store
    vector_store = InMemoryVectorStore(embeddings)
    vector_store.add_documents(splits)
    print("[build_retriever] Added chunks to InMemoryVectorStore.")

    # 4) Retriever 래핑
    retriever = vector_store.as_retriever(search_kwargs={"k": 5})
    return retriever



# Runnable 방식의 RAG 체인 구현

```
query
 → retriever
 → 문서를 하나의 텍스트로 정리
 → ChatPromptTemplate
 → ChatGoogleGenerativeAI(gemini-2.5-flash)
 → StrOutputParser()
 → 최종 답변
```

In [8]:
# ============================================
# 5. RAG 체인 구현 (Gemini LLM)
# ============================================

prompt_template = """
당신은 감사 전문가입니다. 제공된 '관련 감사 사례'를 근거로 사용자의 '질문'에 답변하세요. '관련 감사 사례'가 없거나 적절치 않다면 답변을 통해 사용자에게 알려주세요.

관련 감사 사례: {context}

질문: {question}
"""

prompt = ChatPromptTemplate.from_template(prompt_template)


def format_docs(docs: List[Document]) -> str:
    """리트리버가 찾아준 문서들을 사람이 보기 좋은 하나의 문자열로 합치기"""
    parts = []
    for d in docs:
        title = d.metadata.get("title", "")
        parts.append(f"[{title}]\n{d.page_content}")
    return "\n\n---\n\n".join(parts)

def answer_question(query: str, retriever) -> str:
    """LangChain + Gemini로 RAG 질의 수행 (체인 안 쓰고 수동으로 연결)"""

    # 1) 관련 문서 검색
    docs = retriever.invoke(query)

    # 2) 컨텍스트 문자열로 변환
    context_text = format_docs(docs)

    # 3) 프롬프트 값 생성
    prompt_value = prompt.format(
        context=context_text,
        question=query,
    )

    # 4) Gemini LLM 호출 (ChatGoogleGenerativeAI)
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0.0,
    )

    response = llm.invoke(prompt_value)

    # 5) 최종 텍스트만 리턴
    return response.content



# 테스트

Retriever → 프롬프트 → Gemini → 최종 RAG 답변 출력

In [9]:
# ============================================
# 6. TEST QUERY
# ============================================

def main():
    docs = load_and_prepare_docs(DATA_PATH)
    retriever = build_retriever(docs)

    test_query = "병가기간 중 해외여행을 한 직원에 대한 적정 감사처분과 그 이유를 말해줘"

    print("[TEST QUERY]")
    print(test_query)
    print("\n[ANSWER]\n")

    answer = answer_question(test_query, retriever)
    print(answer)

if __name__ == "__main__":
    main()


[load_and_prepare_docs] Loading from: c:\Users\jskim\aiffel\.aura\JAESEONG\meta_data_contents_summary.json
[load_and_prepare_docs] Created 4961 summary-based documents.
[build_retriever] Split into 6477 chunks.
[build_retriever] Added chunks to InMemoryVectorStore.
[TEST QUERY]
병가기간 중 해외여행을 한 직원에 대한 적정 감사처분과 그 이유를 말해줘

[ANSWER]

제공해주신 '관련 감사 사례'들은 직원이 병가 기간 중 해외여행을 한 경우의 적정 감사 처분 및 그 이유에 대한 내용을 포함하고 있지 않습니다.

제공된 감사 사례들은 다음과 같은 주제를 다루고 있습니다:
*   자회사 채용 실태 점검 및 특별채용 규정 개선
*   개발사업 규정, 지반조사, 공사기간 산정, 입주자 모집 시점 등 건설 및 주택 공급 관련 규정
*   예산 집행 지침 및 불용 조치
*   정보시스템 보안 기본원칙 및 기준
*   계약 업무 처리 시 변경 절차 및 기준

따라서, 제시된 감사 사례만으로는 병가 기간 중 해외여행을 한 직원에 대한 적절한 감사 처분과 그 이유를 판단하여 답변해 드릴 수 없습니다. 해당 사안은 복무 규정, 징계 규정, 또는 관련 판례 등을 검토해야 할 것입니다.
